<a href="https://colab.research.google.com/github/amranzoabi/Cloud-Computing---Alligators-Team/blob/main/HW2_HomePageUser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')
#download files from this link: https://drive.google.com/drive/folders/1L_hvXxcJxhSFQCHwSkt2I-rXql44csiR?usp=share_link
#upload the files to your google drive under directory Colab Notebooks/CloudRacer (create a folder called CloudRacer inside Colab Notebooks and put the files inside it)
#run this before running the code

Mounted at /content/drive


In [3]:
import ipywidgets as widgets
import IPython
from IPython.core.display import display, HTML
from google.colab import output
import json
from firebase import firebase
from IPython.display import Math, HTML, display
import uuid
from IPython.display import Javascript
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import base64
import io

 
cssH = open('/content/drive/MyDrive/Colab Notebooks/CloudRacer/style.css', 'r') #home css file
cssHContent = cssH.read() #home css content


htmlH = open('/content/drive/MyDrive/Colab Notebooks/CloudRacer/index.html', 'r') #home html file
htmlHContent = htmlH.read() #home html content

cssT = open('/content/drive/MyDrive/Colab Notebooks/CloudRacer/cssTutorial.css', 'r') #tutorial css file
cssTContent = cssT.read() #tutorial css content

htmlT = open('//content/drive/MyDrive/Colab Notebooks/CloudRacer/tutorial.html', 'r') #tutorial html file
htmlTContent = htmlT.read() #tutorial html content

cssP = open ('/content/drive/MyDrive/Colab Notebooks/CloudRacer/cssPlay.css', 'r') #play css file
cssPContent = cssP.read() #play css content



def loadPage(pageTitle):
  output.clear()
  if pageTitle == 'home':
    cssContent = cssHContent
    htmlContent = htmlHContent
    display(HTML(f"<style>{cssContent}</style>"))
    display(HTML(f"<html><head></head><body>{htmlContent}</body></html>"))
  if pageTitle == 'tutorial':
    cssContent = cssTContent
    htmlContent = htmlTContent
    display(HTML(f"<style>{cssContent}</style>"))
    display(HTML(f"<html><head></head><body>{htmlContent}</body></html>"))
  if pageTitle == 'play':
    cssContent = cssPContent
    display(HTML(f"<style>{cssContent}</style>"))
    #---------------------------------------PLAY CODE START --------------------------------------------------#
    # Initialize Firebase connection
    FBconn = firebase.FirebaseApplication('https://pythonracer-e7312-default-rtdb.europe-west1.firebasedatabase.app/', None)
    # Load questions from the database
    res = FBconn.get('/Questions', None)
    # Add the function to the button's onclick attribute
    retry_button_html = """<button class="button-1" onclick="google.colab.kernel.invokeFunction('notebook.restart_quiz', [], {});">Retry Quiz</button>"""
    # Function to plot the pie chart and return it as a base64 encoded string
    def plot_pie_chart(correct, incorrect):
      # Prepare data for the pie chart
      labels = ['Correct', 'Incorrect']
      sizes = [correct, incorrect]
      colors = ['#66b3ff','#ff9999']  # light blue and light red
      explode = (0.1, 0)  # explode 1st slice

      # Create the pie chart
      fig1, ax1 = plt.subplots(figsize=(10, 7))  # Increase the figure size
      wedges, texts, autotexts = ax1.pie(sizes, explode=explode, labels=labels, colors=colors, 
            autopct='%1.1f%%', pctdistance=0.85, startangle=90)  # Remove the shadow and move the percentage labels

      # Draw a circle at the center 
      centre_circle = patches.Circle((0,0),0.70,fc='white')
      ax1.add_artist(centre_circle)

      # Add a legend
      plt.legend(labels, loc="best")

      #the font size and style
      plt.rcParams['font.size'] = 16
      plt.rcParams['font.family'] = 'sans-serif'

      # Change the color and weight of the percentage labels
      plt.setp(autotexts, color='black', weight="bold")
    
      # Add a title
      plt.title('Percentage of Correct Answers', fontsize=16, color='black')

      # Save the plot to a BytesIO object
      pic_IObytes = io.BytesIO()
      plt.savefig(pic_IObytes,  format='png')
      pic_IObytes.seek(0)
      pic_hash = base64.b64encode(pic_IObytes.read())
    
      plt.close(fig1)  # Close the figure after saving it

      # Equal aspect ratio ensures that pie is drawn as a circle.
      ax1.axis('equal')

      return "data:image/png;base64,{}".format(pic_hash.decode('utf-8'))



    def restart_quiz():
      # reDefine global variables
      global index, question, answer, correct_ans, num_correct
      index = -1
      question = None
      answer = [None] * 4
      correct_ans = None
      num_correct = 0;
      getNextQuestion()


    def increment_num_correct():
      global num_correct
      num_correct += 1

    def invoke_button(title, callback):
      callback_id = 'button-' + str(uuid.uuid4())
      display(HTML(f'<button name="nextbtn" id="{callback_id}" style="margin: auto; margin-top: 20px; background-color: #EA4C89; border-radius: 8px; border-style: none; box-sizing: border-box; color: #FFFFFF; cursor: pointer; display:none; font-family: &quot;Haas Grot Text R Web&quot;, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 14px; font-weight: 500; height: 40px; line-height: 20px; list-style: none; outline: none; padding: 10px 16px; position: relative; text-align: center; text-decoration: none; transition: color 100ms; vertical-align: baseline; user-select: none; -webkit-user-select: none; touch-action: manipulation;">{title}</button>'))

      output.register_callback(callback_id, callback)
      display(Javascript(f'''
        document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
        }};
      '''))

    def invoke_All():
      invoke_button('next',getNextQuestion)
      # Register the function so it can be called from Javascript
      output.register_callback('notebook.restart_quiz', restart_quiz)
      # Register the variable so it can be called from Javascript
      output.register_callback('notebook.increment_num_correct', increment_num_correct)


    def getNextQuestion():
      global question, answer, correct_ans, index ,num_correct
      index += 1
      # Hide the nextbtn button
      hide_button_js = '''
      var nextshow = document.getElementsByName("nextbtn");
      nextshow[0].style.display = "none";
      '''
      display(IPython.display.Javascript(hide_button_js))
      if index >= len(res):
        correct = num_correct
        incorrect = index - correct
        img_str = plot_pie_chart(correct, incorrect)
        # Quiz Results
        quiz_results = f"""
      
          <h2 id='title'>Congratulations!</h2>
          <div id='quiz'>
          <p id='finalres'>You have successfully completed the Race! 🎉</p>
          <p id='finalres'>Your Score: <strong>{num_correct}/{index}</strong></p>
        

        <div style="text-align:center; margin-top: 30px;">
          <img src='{img_str}' style="width: 500px; height: 500px; object-fit: cover; border-radius: 40%; box-shadow: 0px 5px 15px rgba(0,0,0,0.1);"/>
        </div>

       
          <p id='finalres'>Keep practicing to improve your score!</p>
          {retry_button_html}
        </div>
        """
        quiz_widget.value = quiz_results
        return
      question = None
      while not question:
        key = list(res.keys())[index]
        for x in range(4):
            answer[x] = res[key]["Ans"+str(x+1)]
        question = res[key]["Question"]
        correct_ans = res[key]["Correct"]
      # Define the HTML code for the question and options
      question_html = f"<div  id='title'>CloudRacer</div><br><br><div id='question'>{question}</div>"
      option_html = "<ul id='options'>"
      for i, ans in enumerate(answer):
        option_html += f"<li style='margin-bottom: 10px;'><label class='label'><span class='number'>{i+1}</span><input type='radio' name='option' value='{i}' style='margin-right: 15px;'>{ans}</label></li>"
      option_html += "</ul>"
      # Create a result text element
      result_html = "<div id='result'></div>"
      # Create a button from HTML code
      button_html = "<button id='subbtn' type='button' class='button-1' onclick='myFunction()'>Submit</button>"
      button_close = "<button id='exitbtn' type='button' class='button-1' onclick='exit()' style='margin-left: 5px;'>Exit</button>"
    
      # Create an HTML widget to display the question, options, and button
      quiz_widget.value = f"<div id='quiz'>{button_close}{question_html}{option_html}{result_html}{button_html}</div>"
      # Create a JavaScript function to handle the quiz submission
      js_function = '''
      function myFunction() {
        var nextshow = document.getElementsByName('nextbtn')[0];
        nextshow.style.display = "none";
        var subbtn = document.getElementById('subbtn');
        subbtn.style.display = "block";
        let selected = document.querySelector('input[name="option"]:checked');
        let result = document.querySelector("#result");
        if (selected) {
          let x = parseInt(selected.value) + 1;
          if (x == %s){
            result.textContent = "Correct!";
            result.style.color = "green";
            nextshow.style.display = "block";
            subbtn.style.display = "none";
            google.colab.kernel.invokeFunction('notebook.increment_num_correct', [], {});
          }
          else{
            result.textContent = "Wrong!";
            result.style.color = "red";
            nextshow.style.display = "block";
            subbtn.style.display = "none";
          }
        }
        else {
          result.textContent = "Please select an answer.";
          result.style.color = "orange";
          subbtn.style.margin = "auto";
        }
      }
      function exit(){
        google.colab.kernel.invokeFunction('notebook.loadPage', ['home'], {});
      }
      ''' % (correct_ans)

      # Display the JS function
      display(IPython.display.Javascript(js_function))
    # Create an HTML widget to display the quiz
    quiz_widget = widgets.HTML()
    #invoke all buttons and labels neeeded for the conection between java and python
    restart_quiz()
    # Display the quiz widget
    display(quiz_widget)
    invoke_All()
    #---------------------------------------PLAY CODE END --------------------------------------------------#
    # Start the quiz by displaying the first question


loadPage('home')

output.register_callback('notebook.loadPage', loadPage)